In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

In [2]:
def curate(df):

    try:
        df = df[df['PROVIDER_ID'] != np.nan]
        df['PROVIDER_ID'] = df['PROVIDER_ID'].values.astype(str)
        
        ids = df['PROVIDER_ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['PROVIDER_ID'] = ids2
        
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    if 'Unnamed: 0' in list(df):
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    return df

## MEASURES USED FOR JULY 2022 File: From PDFs (SAS & CMS)

### Mortality

MORT-30-AMI  
MORT-30-CABG  
MORT-30-COPD  
MORT-30-HF  
MORT-30-PN  
MORT-30-STK  
PSI-4-SURG-COMP  

### Safety of Care

HAI-1  
HAI-2  
HAI-3  
HAI-4  
HAI-5  
HAI-6  
COMP-HIP-KNEE  
PSI-90-Safety  

### Readmission

READM-30-CABG  
READM-30-COPD  
READM-30-Hip-Knee  
READM-30-HOSP-WIDE  
EDAC-30-AMI  
EDAC-30-HF  
EDAC-30-PN  
OP-32  
OP-35 ADM  
OP-35 ED  
OP-36  

### Patient Experience

H-COMP-1  
H-COMP-2  
H-COMP-3  
H-COMP-5  
H-COMP-6  
H-COMP-7  
H-CLEAN-HSP / H-QUIET-HSP  
H-HSP-RATING / H-RECMND  

### Timely and Effective Care

IMM-3  
OP-10  
OP-13  
OP-18b  
OP-2  (<= 100 hospitals reporting)  
OP-22  
OP-23  
OP-29
OP-3b  
OP-8  
PC-01  
SEP-1  


## Retired (2022):
ED-2b  
OP-30 

## Retired in (2023):
OP-33: Percentage of patients receiving appropriate radiation therapy for cancer that has
spread to the bone

## Measure excluded from 2023 due to no more than 100 hospitals reporting performance publicly:
OP-2: Percentage of outpatients with chest pain or possible heart attack who got drugs to
break up blood clots within 30 minutes of arrival

## New measure added for 2023:
HCP COVID-19: COVID-19 Vaccination Coverage Among Health Care Providers


# Use 2022 SAS output files to get the bulk of necessary column labels 

In [3]:
sas_input_df_2022 = pd.read_sas(stars_dir + 'Reproduce_Stars_Input/2022/Input_file/all_data_2022jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")


sas_cols_2022 = list(sas_input_df_2022)

print(len(sas_cols_2022), 'columns in 2022 Stars SAS input file (not all get used by the SAS programs):')
print(sas_cols_2022, '\n')

print('Create columns for 2023 data')
print('Adding HCP_COVID_19 to list of column labels\n')
sas_cols1 = sas_cols_2022 + ['HCP_COVID_19']
print('Removing OP_33 and OP_33_DEN')
sas_cols1.remove('OP_33')
sas_cols1.remove('OP_33_DEN')
sas_cols = list(sas_cols1)

sas_input_df_2022 = curate(sas_input_df_2022)
prvdrs_2022 = sas_input_df_2022['PROVIDER_ID'].unique().tolist()

print(len(prvdrs_2022), 'hospitals in 2022 Stars output file\n')
prvdrs_2022 = sorted(list(set(prvdrs_2022)))

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')


95 columns in 2022 Stars SAS input file (not all get used by the SAS programs):
['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_2', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_S

## HAIs

In [4]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Healthcare_Associated_Infections-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['HAI_1_ELIGCASES', 'HAI_1_DOPC', 'HAI_1_SIR', 'HAI_2_ELIGCASES', 'HAI_2_DOPC', 'HAI_2_SIR', 
            'HAI_3_ELIGCASES', 'HAI_3_DOPC', 'HAI_3_SIR', 'HAI_4_ELIGCASES', 'HAI_4_DOPC', 'HAI_4_SIR', 
            'HAI_5_ELIGCASES', 'HAI_5_DOPC', 'HAI_5_SIR', 'HAI_6_ELIGCASES', 'HAI_6_DOPC', 'HAI_6_SIR']

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

hai_df = pd.DataFrame(columns=['Facility ID']) 
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    hai_df = hai_df.merge(tdf2, on='Facility ID', how='outer')
    
hai_df.rename(columns={'HAI_1_ELIGCASES': 'HAI_1_DEN_PRED',
                       'HAI_1_DOPC': 'HAI_1_DEN_VOL',
                       'HAI_1_SIR': 'HAI_1',
                       'HAI_2_ELIGCASES': 'HAI_2_DEN_PRED',
                       'HAI_2_DOPC': 'HAI_2_DEN_VOL',
                       'HAI_2_SIR': 'HAI_2',
                       'HAI_3_ELIGCASES': 'HAI_3_DEN_PRED',
                       'HAI_3_DOPC': 'HAI_3_DEN_VOL',
                       'HAI_3_SIR': 'HAI_3',
                       'HAI_4_ELIGCASES': 'HAI_4_DEN_PRED',
                       'HAI_4_DOPC': 'HAI_4_DEN_VOL',
                       'HAI_4_SIR': 'HAI_4',
                       'HAI_5_ELIGCASES': 'HAI_5_DEN_PRED',
                       'HAI_5_DOPC': 'HAI_5_DEN_VOL',
                       'HAI_5_SIR': 'HAI_5',
                       'HAI_6_ELIGCASES': 'HAI_6_DEN_PRED',
                       'HAI_6_DOPC': 'HAI_6_DEN_VOL',
                       'HAI_6_SIR': 'HAI_6',
                       'Facility ID': 'PROVIDER_ID',
                   }, inplace=True)

for c in list(hai_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
hai_df = curate(hai_df)

75 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'EDAC_30_AMI', 'EDAC_30_AMI_DEN', 'EDAC_30_HF', 'EDAC_30_HF_DEN', 'EDAC_30_PN', 'EDAC_30_PN_DEN', 'HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_32', 'OP_32_DEN', 'OP_35_ADM', 'OP_35_ADM_DEN', 'OP_35_ED', 'OP_35_ED_DEN', 'OP_36', 'OP_36_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY',

## Unplanned Hospital Visits


In [5]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Unplanned_Hospital_Visits-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 
            'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'READM_30_HOSP_WIDE']

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Denominator', 'Measure ID', 'Score'], axis=1)

uhv_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    uhv_df = uhv_df.merge(tdf2, on='Facility ID', how='outer')

uhv_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(uhv_df):
    try:
        sas_cols.remove(c)
    except:
        pass
    
print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
uhv_df = curate(uhv_df)

53 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 'SEP_1', 'SEP_1_DEN'] 



## COMPLICATIONS AND DEATHS

In [6]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Complications_and_Deaths-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 
            'MORT_30_PN', 'MORT_30_STK', 'PSI_04', 'COMP_HIP_KNEE',
            'PSI_90', 
            ]

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Denominator'], axis=1)

cad_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    cad_df = cad_df.merge(tdf2, on='Facility ID', how='outer')
    
cad_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'PSI_04': 'PSI_4_SURG_COMP',
                       'PSI_04_DEN': 'PSI_4_SURG_COMP_DEN',
                       'PSI_90': 'PSI_90_SAFETY',
                       'PSI_90_DEN': 'PSI_90_SAFETY_DEN',
                   }, inplace=True)

for c in list(cad_df):
    try:
        sas_cols.remove(c)
    except:
        pass
    
print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
cad_df = curate(cad_df)

35 remaining features: ['HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'SEP_1', 'SEP_1_DEN'] 



## TIMELY AND EFFECTIVE CARE

Everything except PC-01, which for 2023 is located in the Maternal Health files of the hospitals data archive

In [7]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Timely_and_Effective_Care-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique(), '\n')

measures = ['IMM_3', 'OP_18b', 'OP_2', 'OP_22', 'OP_23', 'OP_29', 'OP_3b', 'SEP_1', 'HCP_COVID_19']

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Sample', 'Measure ID', 'Score'], axis=1)

tec_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec_df = tec_df.merge(tdf2, on='Facility ID', how='outer')
    
tec_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                       'OP_3b': 'OP_3B', 
                       'OP_3b_DEN': 'OP_3B_DEN', 
                       'OP_18b': 'OP_18B',
                       'OP_18b_DEN': 'OP_18B_DEN',
                      }, inplace=True)

for c in list(tec_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
tec_df = curate(tec_df)


['EDV' 'ED_2_Strata_1' 'ED_2_Strata_2' 'HCP_COVID_19' 'IMM_3' 'OP_18b'
 'OP_18c' 'OP_2' 'OP_22' 'OP_23' 'OP_29' 'OP_31' 'OP_3b'
 'SAFE_USE_OF_OPIOIDS' 'SEP_1' 'SEP_SH_3HR' 'SEP_SH_6HR' 'SEV_SEP_3HR'
 'SEV_SEP_6HR' 'STK_02' 'STK_03' 'STK_05' 'STK_06' 'VTE_1' 'VTE_2'] 

18 remaining features: ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN'] 



In [8]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/VA_TE.csv')

measures = ['IMM-3', 'OP-18b', 'OP-2', 'OP-22', 'OP-23', 'OP-29', 'OP-3b', 'SEP-1', 'HCP-COVID-19']

tec3_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec3_df = tec3_df.merge(tdf2, on='Facility ID', how='outer')
    
tec3_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                        'OP-3b': 'OP_3B',
                        'OP-3b_DEN': 'OP_3B_DEN',
                        
                        'OP-18b': 'OP_18B',
                        'OP-18b_DEN': 'OP_18B_DEN',
                        
                        'IMM-3': 'IMM_3', 
                        'IMM-3_DEN': 'IMM_3_DEN', 
                        
                        'OP-2': 'OP_2', 
                        'OP-2_DEN': 'OP_2_DEN', 
                        
                        'OP-22': 'OP_22', 
                        'OP-22_DEN': 'OP_22_DEN', 
                        
                        'OP-23': 'OP_23', 
                        'OP-23_DEN': 'OP_23_DEN', 
                        
                        'OP-29': 'OP_29', 
                        'OP-29_DEN': 'OP_29_DEN', 
                        
                        'SEP-1': 'SEP_1', 
                        'SEP-1_DEN': 'SEP_1_DEN', 
                        
                        'HCP-COVID-19': 'HCP_COVID_19',
                        'HCP-COVID-19_DEN': 'HCP_COVID_19_DEN',
                        
                      }, inplace=True)

tec3_df.replace('Not Available', np.nan, inplace=True)
print(tec3_df.shape)
tec3_df = curate(tec3_df)
print(tec3_df.shape)

print(tec_df.shape)
print(tec3_df.shape)
tec_df = pd.concat([tec_df, tec3_df])
print(tec_df.shape)

tec_df.head()

(137, 19)
(137, 19)


## TIMELY AND EFFECTIVE CARE

For 2023, PC-01 is located in the Maternal Health files of the hospitals data archive

In [10]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Maternal_Health-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique())

measures = ['PC_01']
df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Sample'], axis=1)

tec2_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    tec2_df = tec2_df.merge(tdf2, on='Facility ID', how='outer')
    
tec2_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(tec2_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')

tec2_df = curate(tec2_df)


['PC_01' 'PC_05' 'SM_7']
16 remaining features: ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## HCAHPS

In [11]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/HCAHPS-Hospital.csv')
print(df['HCAHPS Measure ID'].unique())

measures = ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 
            'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 
            'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING']

df = df[df['HCAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'HCAHPS Measure ID', 'Patient Survey Star Rating', 
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

HCAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['HCAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Patient Survey Star Rating'].tolist()
    if i == 0:
        tdf2['H_NUMB_COMP'] = tdf1['Number of Completed Surveys'].tolist()
        tdf2['H_RESP_RATE_P'] = tdf1['Survey Response Rate Percent'].tolist()
        
    HCAHPS_df = HCAHPS_df.merge(tdf2, on='Facility ID', how='outer')
    

HCAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'] = HCAHPS_df['H_HSP_RATING_STAR_RATING'].astype(int)
HCAHPS_df['H_RECMND_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_RECMND_STAR_RATING'] = HCAHPS_df['H_RECMND_STAR_RATING'].astype(int)
HCAHPS_df['H_GLOB_STAR_RATING'] = np.round((HCAHPS_df['H_HSP_RATING_STAR_RATING'] + HCAHPS_df['H_RECMND_STAR_RATING']) / 2, 1)
HCAHPS_df['H_GLOB_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df['H_CLEAN_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_CLEAN_STAR_RATING'] = HCAHPS_df['H_CLEAN_STAR_RATING'].astype(int)
HCAHPS_df['H_QUIET_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_QUIET_STAR_RATING'] = HCAHPS_df['H_QUIET_STAR_RATING'].astype(int)
HCAHPS_df['H_INDI_STAR_RATING'] = np.round((HCAHPS_df['H_CLEAN_STAR_RATING'] + HCAHPS_df['H_QUIET_STAR_RATING']) / 2, 1)
HCAHPS_df['H_INDI_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df.drop(labels = ['H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING'], axis=1, inplace=True)

for c in list(HCAHPS_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
HCAHPS_df = curate(HCAHPS_df)

['H_COMP_1_A_P' 'H_COMP_1_SN_P' 'H_COMP_1_U_P' 'H_COMP_1_LINEAR_SCORE'
 'H_COMP_1_STAR_RATING' 'H_NURSE_RESPECT_A_P' 'H_NURSE_RESPECT_SN_P'
 'H_NURSE_RESPECT_U_P' 'H_NURSE_LISTEN_A_P' 'H_NURSE_LISTEN_SN_P'
 'H_NURSE_LISTEN_U_P' 'H_NURSE_EXPLAIN_A_P' 'H_NURSE_EXPLAIN_SN_P'
 'H_NURSE_EXPLAIN_U_P' 'H_COMP_2_A_P' 'H_COMP_2_SN_P' 'H_COMP_2_U_P'
 'H_COMP_2_LINEAR_SCORE' 'H_COMP_2_STAR_RATING' 'H_DOCTOR_RESPECT_A_P'
 'H_DOCTOR_RESPECT_SN_P' 'H_DOCTOR_RESPECT_U_P' 'H_DOCTOR_LISTEN_A_P'
 'H_DOCTOR_LISTEN_SN_P' 'H_DOCTOR_LISTEN_U_P' 'H_DOCTOR_EXPLAIN_A_P'
 'H_DOCTOR_EXPLAIN_SN_P' 'H_DOCTOR_EXPLAIN_U_P' 'H_COMP_3_A_P'
 'H_COMP_3_SN_P' 'H_COMP_3_U_P' 'H_COMP_3_LINEAR_SCORE'
 'H_COMP_3_STAR_RATING' 'H_CALL_BUTTON_A_P' 'H_CALL_BUTTON_SN_P'
 'H_CALL_BUTTON_U_P' 'H_BATH_HELP_A_P' 'H_BATH_HELP_SN_P'
 'H_BATH_HELP_U_P' 'H_COMP_5_A_P' 'H_COMP_5_SN_P' 'H_COMP_5_U_P'
 'H_COMP_5_LINEAR_SCORE' 'H_COMP_5_STAR_RATING' 'H_MED_FOR_A_P'
 'H_MED_FOR_SN_P' 'H_MED_FOR_U_P' 'H_SIDE_EFFECTS_A_P'
 'H_SIDE_EFFECTS_SN_P'

## Outpatient Imaging Efficiency

The July 2023 Overall Star Ratings were calculated using the measure data from the January 2023 Care Compare update, with the re-released OP-13 measure data publicly reported in April 2023 on Care Compare.

Make two dataframes: 1 for OP-13 (April 2023) and 1 for other measures (Jan 2023)

In [12]:
df = pd.read_csv(hos_dir + '2023/hospitals_04_2023/Outpatient_Imaging_Efficiency-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique())

measures = ['OP-13']
df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-13': 'OP_13',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
oie_df1 = curate(oie_df)


['OP-10' 'OP-13' 'OP-39' 'OP-8']
5 remaining features: ['OP_10', 'OP_10_DEN', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



In [13]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Outpatient_Imaging_Efficiency-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique())

measures = ['OP-8', 'OP-10']
df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-8': 'OP_8',
                       'OP-10': 'OP_10',
                       'OP-13': 'OP_13',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
oie_df2 = curate(oie_df)


['OP-10' 'OP-13' 'OP-39' 'OP-8']
3 remaining features: ['OP_10_DEN', 'OP_13_DEN', 'OP_8_DEN'] 



## MERGE DATAFRAMES

In [14]:
main_df = tec_df.merge(tec2_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(cad_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(HCAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(uhv_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(hai_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(oie_df2, on='PROVIDER_ID', how='outer')

prvdrs = main_df['PROVIDER_ID'].unique().tolist()
oie_df1 = oie_df1[oie_df1['PROVIDER_ID'].isin(prvdrs)]
main_df = main_df.merge(oie_df1, on='PROVIDER_ID', how='outer')

main_df['OP_10_DEN'] = np.nan
main_df['OP_13_DEN'] = np.nan
main_df['OP_8_DEN'] = np.nan

for c in list(main_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
print(main_df.shape)

for col in list(main_df):
    if col != 'PROVIDER_ID':
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce')


0 remaining features: [] 

(4848, 95)


In [15]:

# Remove children's hospitals
tdf = main_df.copy(deep=True)
prvdrs1 = tdf['PROVIDER_ID'].unique().tolist()
prvdrs2 = []
for p in prvdrs1:
    
    try:
        p_suf = float(p[2:])

        if p_suf < 3300 or p_suf > 3399:
            prvdrs2.append(p)
            
    except:
        prvdrs2.append(p)
        
tdf = tdf[tdf['PROVIDER_ID'].isin(prvdrs2)]

# Remove hospitals containing no data for overall star rating measures
tdf = tdf.loc[:, ~tdf.columns.str.contains('_DEN')]
ls = list(tdf)
ls.remove('PROVIDER_ID')
tdf.dropna(how='all', axis=0, subset=ls, inplace=True)

main_df = main_df[main_df['PROVIDER_ID'].isin(tdf['PROVIDER_ID'].unique().tolist())]
print(main_df.shape)

ls = np.setdiff1d(list(main_df), sas_cols1)
main_df = main_df.filter(items=sas_cols1, axis=1)
print('main_df.shape:', main_df.shape)


(4687, 95)
main_df.shape: (4687, 94)


## Multiple select features by 0.01

Several features in the SAS file are represented as rates on the scale of 0 to 1, but are represented in the hospitals data archive as percentages on the scale of 0 to 100.

In [16]:
ls = list(main_df)
ls.remove('PROVIDER_ID')

prvdrs_main1 = sorted(main_df['PROVIDER_ID'].tolist())

ls = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
      'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
      'OP_23', 'OP_29', 'IMM_3', 'PC_01', 'SEP_1', 'MORT_30_CABG',
      'READM_30_CABG', 'READM_30_HOSP_WIDE', 'OP_2', 'OP_8',
      'OP_10', 'OP_13', 'HCP_COVID_19']

for l in ls: 
    main_df[l] = main_df[l] * 0.01


## Prepare data for import into SAS as csv

**Issue:** Python can read SAS files but it can't write them. Only SAS can write SAS files. Consequently, the dataframe produced here must be exported as a CSV file. When importing the CSV into SAS, SAS has a problem interpreting the CMS IDs of hospitals because it tries to interpret some IDs as numbers and others as strings (character variables). This is because VHA hospitals contain an 'F' at the end of their IDs.

**Solution:** Remove the 'F' suffix from VHA hospitals and replaced it with 666666. Doing so alleviate the issue of importing the CSV file and not causing in error in reading CMS IDs, while preventing ID conflicts.

In [17]:
prvdrs = []
for p in main_df['PROVIDER_ID'].tolist():
    if 'F' in p:
        p = p[:-1]
        p = p + '666666'
    prvdrs.append(p)

main_df['PROVIDER_ID'] = prvdrs

print(list(main_df))
main_df.head()


['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_2', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'IMM_3_DEN', 'IMM_3', 'PC_01'

,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_2,OP_2_DEN,OP_3B,OP_3B_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,HCP_COVID_19
0,010001,10024.0,17731.0,154.0,200.0,101908.0,101451.0,10.597,26.630,4.548,1.845,9.412,72.686,0.661,0.300,1.099,0.0,0.850,0.660,0.142,0.042,21.8,0.199,1.9,-1.5,0.164,0.159,0.083,0.085,0.124,0.024,3058.0,98.0,755.0,202.0,319.0,436.0,489.0,407.0,630.0,182.0,317.0,102.0,NaN,NaN,NaN,NaN,0.425,NaN,0.057,NaN,0.067,NaN,205.0,323.0,0.03,51079.0,NaN,NaN,0.81,16.0,173.39,120.0,1.01,3795.0,0.97,0.09,34.0,0.46,146.0,15.0,2.0,3.0,2.0,4.0,4.0,3.0,3.0,3.5,434.0,NaN,0.047,172.0,0.117,165.0,14.1,254.0,10.2,214.0,4.7,214.0,1.0,688.0,0.737
1,010005,3713.0,8670.0,88.0,38.0,38413.0,35686.0,2.450,4.995,2.512,0.316,1.999,10.484,3.673,1.201,1.194,NaN,0.000,0.858,0.139,0.040,9.3,0.176,4.7,2.2,0.166,0.218,0.169,0.081,0.126,0.018,1258.0,178.0,157.0,234.0,38.0,361.0,100.0,369.0,153.0,195.0,52.0,134.0,NaN,NaN,NaN,NaN,0.545,NaN,0.138,NaN,0.043,NaN,146.0,1003.0,0.03,54503.0,0.73,15.0,0.99,108.0,142.88,35.0,0.91,2593.0,0.90,0.01,194.0,0.59,242.0,16.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,3.5,717.0,NaN,NaN,NaN,NaN,NaN,14.6,850.0,11.1,108.0,5.7,108.0,0.9,362.0,0.821
2,010006,7318.0,11755.0,91.0,35.0,62709.0,54159.0,7.924,15.296,2.523,0.373,4.164,22.618,0.757,0.196,0.396,NaN,1.441,0.088,0.142,0.048,-2.3,0.177,25.9,42.3,0.189,0.178,0.122,0.078,0.165,0.034,2555.0,246.0,550.0,235.0,312.0,538.0,261.0,528.0,468.0,209.0,295.0,234.0,NaN,NaN,NaN,NaN,0.412,NaN,0.110,NaN,0.014,NaN,144.0,363.0,0.01,41137.0,0.57,14.0,0.88,75.0,157.42,84.0,1.10,2292.0,0.64,0.00,37.0,0.58,142.0,17.0,2.0,3.0,1.0,2.0,3.0,2.0,2.0,2.5,1358.0,NaN,0.035,117.0,0.156,109.0,12.5,1505.0,NaN,NaN,NaN,NaN,1.1,468.0,0.651
3,010007,268.0,1417.0,6.0,NaN,5484.0,5413.0,0.165,0.774,0.152,NaN,0.132,2.148,NaN,NaN,NaN,NaN,NaN,0.466,0.151,NaN,36.3,0.197,NaN,-12.6,NaN,0.217,0.139,0.103,NaN,NaN,272.0,NaN,51.0,72.0,NaN,99.0,NaN,106.0,45.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059,NaN,NaN,NaN,119.0,1202.0,0.03,11120.0,NaN,NaN,0.63,68.0,NaN,NaN,0.99,318.0,0.61,NaN,NaN,0.93,55.0,23.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,173.0,NaN,NaN,NaN,NaN,NaN,15.3,118.0,NaN,NaN,NaN,NaN,1.0,56.0,0.574
4,010008,14.0,488.0,NaN,NaN,2171.0,2171.0,0.008,0.265,NaN,NaN,0.051,0.398,NaN,NaN,NaN,NaN,NaN,NaN,0.145,NaN,NaN,NaN,NaN,NaN,NaN,0.197,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021,NaN,NaN,NaN,113.0,346.0,0.00,6205.0,NaN,NaN,0.52,23.0,NaN,NaN,NaN,125.0,0.46,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,14.3,62.0,NaN,NaN,NaN,NaN,NaN,NaN,0.623


In [18]:
main_df['PROVIDER_ID'] = pd.to_numeric(main_df['PROVIDER_ID'], errors='coerce')
main_df.sort_values(by=['PROVIDER_ID'], ascending = True, inplace = True)
main_df.to_csv(stars_dir + "Reproduce_Stars_Input/2023/Input_File/all_data_2023jul.csv", index=False)
